<a href="https://colab.research.google.com/github/Jofdiazdi/TalleresSimpy/blob/master/Talleres/2.%20Simulacion%20de%20eventos%20discretos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulación en Python usando Simpy: cola simple

---
Ahora veremos un ejemplo de simpy para la simulacion de una cola simple.

El enunciado es el siguiente:
 
Una peluquería tiene un peluquero que se demora entre 15 y 30 minutos por corte. La peluquería recibe en promedio 3 clientes por hora (es decir, uno cada 20 minutos). Se desea simular las llegadas y servicios de 5 clientes. 

Una vez realizado el código, podremos variar cualquiera de éstos parámetros (tener varios peluqueros, cambiar el tiempo que se demoran por corte, simular para n clientes o para un tiempo específico como una cierta cantidad de minutos, etc).

Antes de empezar primero verificamos que simpy este instalado:


In [0]:
# EJECUTAR PARA INSTALAR simpy
!pip install simpy

## Simulación de eventos discretos


Primero se deben importar las librerias que vamos a usar:

**random**: Libreria que nos ayuda a generar numeros aleatoreos con las diferentes distribuciones.

**math**: Libreria matematica de python

In [0]:
# Cargando las bibliotecas a utilizar
import random   # Para generar números pseudoaleatorios
import math     # Para la funcion logaritmo
import simpy    # Proporciona un entorno de simulacion

Primero definimos las variables que vamos a utilizar:

In [0]:
# Definoendo las constantes y variables a utilizar
SEMILLA = 30                #Semilla para replicar el comportamiento del sistema
NUM_PELUQUEROS = 1          # Numero de peluqueros en la peliqueria
TIEMPO_CORTE_MIN = 15       # Tiempo minimo de corte
TIEMPO_CORTE_MAX = 30       # Tiempo maximo de corte
T_LLEGADAS = 20             # Tiempo entre llegadas "Lamda"
TIEMPO_SIMULACION = 480     # Tiempo Maximo de simulacion
TOT_CLIENTES = 5            # Numero de clientes a atender

# Variables de desempeño
te  = 0.0   # tiempo de espera total
dt  = 0.0   # duracion de servicio total
fin = 0.0   # minuto en el que finaliza

## Modelando las llegadas de los usuarios

Lo primero que vamos amodelar sera el comportamiento de llegada de los usuarios,segun el enunciado se comprotan asi:

**_Tiempos de llegadas_**: El tiempo entre llegadas de los clientes a la peluquería es exponencial con media de λ=20 minutos. Los clientes son atendidos con base en la disciplina FIFO (primero en llegar, primero en salir), de modo que las llegadas son calculadas de acuerdo a la siguiente fórmula:

T_LLEGADAS = –λ ln(R)

Donde R es un número pseudoaleatorio.

Para esto python nos da la siguiente funcion que nos devuelve un numero aleatorio con distribucion exponencial: `random.expovariate(λ)`

In [0]:
def principal (env, personal):                           #Defnimos la funcion de llegadas
	llegada = 0
	i = 0
	while i<TOT_CLIENTES:                                   #Simulara la llegada de x clientes en este caso 5.
		llegada = random.expovariate(T_LLEGADAS)              # Generamos la proxima llegada con una Distribucion exponencial
		yield env.timeout(llegada)                            # Agendamos la proxmia llegada en el tiempo generado es decir (env.now + llegada)
		i += 1
		env.process(cliente(env, 'Cliente %d' % i, personal)) #Creamos el porceso cliente para que sea ejecutado en el ambiente

## Modelando el servicio de corte

Ahora debemos modelar el comportamiento del servicio, en este caso el corte, el cual se comporta de la sigueinte manera:

**_Tiempos de servicio_**: Los tiempos de servicio son calculados de acuerdo a la siguiente fórmula:

tiempo_corte = TIEMPO_CORTE_MIN + (TIEMPO_CORTE_MAX – TIEMPO_CORTE_MIN)*R

Esto es: el mínimo de tiempo que se demora el peluquero, en nuestro ejemplo es 15, más la diferencia entre el máximo y el mínimo, en nuestro ejemplo serían 15 minutos (30 menos 15), multiplicado por un número pseudoaleatorio. El resultado nos dará un número entre 15 y 30.

Para facilitarnos la vida, la libreria random de python tiene una funcion la cual te da valores uniformes entre 2 numeros: `random.uniform(min,max)`

In [0]:
# Proceso de corte
def cortar(cliente):
	global dt                                           # Para poder acceder a la variable dt declarada anteriormente 
	tiempo_corte = random.uniform(TIEMPO_CORTE_MIN, TIEMPO_CORTE_MAX)       # Distribucion uniforme
	yield env.timeout(tiempo_corte)                     # deja correr el tiempo n minutos
	print(" \o/ Corte listo a %s en %.2f minutos" % (cliente,tiempo_corte))
	dt = dt + tiempo_corte                              # Acumula los tiempos de uso de la i

## Modelado del comportamiento del cliente

El comportamiento del cliente es muy sensillo
1.   El cliente llega a la peluqueria
2.   El cliente pide turno
3.   Si el servidor esta desocupado pasa a ser atendido si no espera su turno
4.   Espera el corte del cabello
5.   Sale de la peluqueria

Tambien en medio del proceso aprovecharemos para ir calculando algunas variables de desempecho, en este caso el tiempo promedio de espera.

In [0]:
# Proceso del cliente
def cliente (env, name, personal ):
	global te
	global fin
	llega = env.now                                     # Llega el cleinte y guardasu hora de llegada
	print ("---> %s llego a peluqueria en minuto %.2f" % (name, llega))
	with personal.request() as request:                 # Pide su turno
		yield request                                     # Espera a ser atendido
		pasa = env.now                                    # Es atendido y guarda el minuto cuado comienza a ser atendido
		espera = pasa - llega                             # Calcula el tiempo que espera -> Calculo del tiempo promedio de espera
		te = te + espera                                  # Acumula los tiempos de espera -> Calculo del tiempo promedio de espera
		print ("**** %s pasa con peluquero en minuto %.2f habiendo esperado %.2f" % (name, pasa, espera))
		yield env.process(cortar(name))                   # Espera a que el corte termine
    personal.release(request)                         #Libera el turno usado
		deja = env.now                                    # Termina el corte y guarda el minuto en que termina el proceso cortar 
		print ("<--- %s deja peluqueria en minuto %.2f" % (name, deja))
		fin = deja                                        # Conserva globalmente el tiempo en el que el servicio termina y el usuario sale del sistema

## Creacion de ambiente y recursos
A continuacion creamos el ambiente en donde se simulara el sistema, creamos el proceso inicial y corremos la simulacion.

### Recursos

Simpy nos ofrece una herramienta mu util a la hora de modelar recursos limitados como en este caso el personal que ofrese el sevicio.

Para ello podemos crear dichos recursos usando:

```
simpy.Resource(env, capacity)
```

Para acceder a ellos debemos usar dentrod el proceso que lo decesita las siguientes linies

```
with {Recurso}.Request() as req:
  yield req
  #Hacer cosas cuando se tenga el turno
  yield env.timeout(tiempo que se uso el recurso)
  {Recurso}.Release(req)
```

Las funciones request() y release() piden y liberan un turno respectivamente es muy importante la linea `yield req` ya que esta nos va a garantizar que se va a esperar el turno.





In [0]:
# PROGRAMA PRINCIPAL    
print ("------------------- Bienvenido Simulacion Peluqueria ------------------")
random.seed (SEMILLA)  # Se inicializa la semilla dle random
env = simpy.Environment() # Crea el entorno o ambiente de simulacion
personal = simpy.Resource(env, NUM_PELUQUEROS) #Crea los recursos (peluqueros)
env.process(principal(env, personal)) #Se crea el proceso que crea los clientes
env.run(until = TIEMPO_SIMULACION) #Inicia la simulacion hasta que no halla mas eventos o se cumpla el tiempo de simulacion

# Calculo de medidas de desempeño
print ("\n---------------------------------------------------------------------")
print ("\nIndicadores obtenidos: ")

lpc = te / fin
print ("\nLongitud promedio de la cola: %.2f" % lpc)
tep = te / TOT_CLIENTES
print ("Tiempo de espera promedio = %.2f" % tep)
upi = (dt / fin) / NUM_PELUQUEROS
print ("Uso promedio de la instalacion = %.2f" % upi)
print ("\n---------------------------------------------------------------------")



------------------- Bienvenido Simulacion Peluqueria ------------------
---> Cliente 1 llego a peluqueria en minuto 12.36
**** Cliente 1 pasa con peluquero en minuto 12.36 habiendo esperado 0.00
 \o/ Corte listo a Cliente 1 en 15.45 minutos
<--- Cliente 1 deja peluqueria en minuto 27.81
---> Cliente 2 llego a peluqueria en minuto 37.17
**** Cliente 2 pasa con peluquero en minuto 37.17 habiendo esperado 0.00
---> Cliente 3 llego a peluqueria en minuto 45.67
 \o/ Corte listo a Cliente 2 en 18.15 minutos
<--- Cliente 2 deja peluqueria en minuto 55.32
**** Cliente 3 pasa con peluquero en minuto 55.32 habiendo esperado 9.65
---> Cliente 4 llego a peluqueria en minuto 72.83
 \o/ Corte listo a Cliente 3 en 20.96 minutos
<--- Cliente 3 deja peluqueria en minuto 76.28
**** Cliente 4 pasa con peluquero en minuto 76.28 habiendo esperado 3.45
---> Cliente 5 llego a peluqueria en minuto 81.70
 \o/ Corte listo a Cliente 4 en 29.83 minutos
<--- Cliente 4 deja peluqueria en minuto 106.11
**** Cliente 

# CONCLUSIÓN

La presente simulación en Python usando Simpy es una herramienta para obtener resultados en forma más sencilla que si lo hiciéramos manualmente. Es tambien muy flexible y se pueden crear programas más elaborados que si utilizáramos una hoja de cálculo.

Python es un lenguaje multiplataforma, ademas, es software libre, por lo que es ideal para cualquier proyecto académico.

Es posible adaptar este programa para resolver diferentes problemas y se puede ampliar para obtener más datos cuya solución manual o matemática pudiera ser complicada.
